In [1]:
from currensee.agents.complete_graph import compiled_graph
from currensee.agents.tools.finance_tools import generate_macro_table
import warnings

warnings.filterwarnings('ignore')

## Define Initial State

This is data that we should be retrieving from each meeting invite.

**DO NOT** change this data until the CRM DB has been updated

In [2]:
init_state = {
        'client_name': 'Adam Clay',
        'client_email': 'adam.clay@compass.com',
        'meeting_timestamp': '2024-03-26 11:00:00',
        'meeting_description': 'Compass - Annual Credit Facility Review Meeting'
    }

In [3]:
# Email prompt summary should be updated in the file 
# currensee/agents/tools/outlook_tools.py in the function produce_client_email_summary

In [4]:
# Finance news prompt summary should be updated in the file 
# currensee/agents/tools/finance_tools.py in the summarize_finance_outputs function

In [5]:
result = compiled_graph.invoke(init_state)

In [6]:
summary = result['final_summary']

# === Add the macro snapshot
macro_table = generate_macro_table()

In [7]:
full_report = summary + "\n\n### Macro Financial Snapshot\n\n" + macro_table
print(full_report)

**Compass - Annual Credit Facility Review Meeting Preparation Document**

**I. Past Meeting/Email Summary**

This meeting pertains to the annual credit facility review for Compass, a leading US residential real estate brokerage.  Our email correspondence reveals that Compass successfully secured a $25 million revolving line of credit from Bankwell Financial.  The process commenced with an initial outreach from Bankwell following a conference meeting, progressing through negotiations culminating in a finalized term sheet.  Internal approvals were obtained by Compass, and subsequently, the definitive loan agreement was drafted and finalized. The credit facility officially closed and funded, and Compass has since utilized the facility through several drawdowns without encountering any reported issues. Currently, the credit facility is operating smoothly, and there are no known problems regarding drawdowns or reporting requirements.

**II. Financial News Data: Relevant Market Conditions an

In [8]:
#Traceback

client_industry_summary = result.get('client_industry_summary', [])
macro_news_summary = result.get('macro_news_summary', [])

# Function to format and print client_industry and macro_news summaries
def print_summary(summary, summary_name):
    print(f"\n{summary_name}:\n{'='*len(summary_name)}")
    for article in summary:
        title = article.get('title', 'No Title')
        snippet = article.get('snippet', 'No Snippet')
        date = article.get('date', 'No Date')
        link = article.get('link', '')
        
        # Extract the source from the domain of the link
        source = link.split("/")[2] if link else 'No Source'
        
        # Format and print the information
        print(f"Title: {title}")
        print(f"Snippet: {snippet}")
        print(f"Date: {date}")
        print(f"Source: {source}")
        print(f"Link: {link}")  # Displaying the full link
        print("-" * 40)  # Just a separator for readability

# Function to format and print the holdings summary
def print_holdings_summary(holdings_summary):
    print("\nClient Holdings Summary:")
    print("========================")

    for holding, articles in holdings_summary.items():
        print(f"\n--- {holding} ---")
        if not articles:
            print("No news found.")
            continue

        for article in articles:
            title = article.get('title', 'No Title')
            snippet = article.get('snippet', 'No Snippet')
            date = article.get('date', 'No Date')
            link = article.get('link', '')
            source = link.split("/")[2] if link else 'No Source'

            print(f"Title: {title}")
            print(f"Snippet: {snippet}")
            print(f"Date: {date}")
            print(f"Source: {source}")
            print(f"Link: {link}")
            print("-" * 40)
            
# Print the formatted summaries
print_summary(client_industry_summary, 'Client Industry Summary')
print_summary(macro_news_summary, 'Macro News Summary')
print_holdings_summary(result.get("client_holdings_summary", {}))


Client Industry Summary:
Title: Compass, Inc. Reports Record First Quarter 2025 Results
Snippet: Revenue in Q1 2025 increased by 28.7% year-over-year to $1.4 billion as transactions increased 27.8% compared to the market transactions, which ...
Date: 5 days ago
Source: finance.yahoo.com
Link: https://finance.yahoo.com/news/compass-inc-reports-record-first-200600921.html
----------------------------------------
Title: Compass is in talks to buy Buffett's real-estate brokerage unit, WSJ ...
Snippet: Home brokerage Compass (COMP.N) is in advanced talks to acquire Warren Buffett's real-estate brokerage business, the Wall Street Journal reported on Thursday.
Date: Mar 13, 2025
Source: www.reuters.com
Link: https://www.reuters.com/markets/deals/compass-is-talks-buy-buffetts-real-estate-brokerage-unit-wsj-reports-2025-03-13/
----------------------------------------
Title: Compass Launches Compass One: The Industry's First-Ever All-in ...
Snippet: Compass (NYSE: COMP), the largest residential

In [9]:
#Retrieving specific figures within the summary

import re
from difflib import SequenceMatcher

# Extract facts from full_report — number + context window
def extract_facts_with_context(text, window=8):
    tokens = text.split()
    facts = []

    for i, word in enumerate(tokens):
        if re.match(r'\$?\d[\d,\.]*(%| billion| million|B|M|€)?\+?', word):
            start = max(i - window, 0)
            end = min(i + window + 1, len(tokens))
            context_phrase = " ".join(tokens[start:end])
            facts.append({
                "value": word.replace(",", "").strip(),
                "phrase": context_phrase
            })

    return facts

# Extract facts from article snippets with the same structure
def extract_snippet_facts(summary):
    snippet_facts = []

    for article in summary:
        snippet = article.get("snippet", "")
        title = article.get("title", "")
        link = article.get("link", "")
        source = article.get("source", "")

        tokens = snippet.split()
        for i, word in enumerate(tokens):
            if re.match(r'\$?\d[\d,\.]*(%| billion| million|B|M|€)?\+?', word):
                start = max(i - 8, 0)
                end = min(i + 8 + 1, len(tokens))
                context = " ".join(tokens[start:end])
                snippet_facts.append({
                    "value": word.replace(",", "").strip(),
                    "phrase": context,
                    "snippet": snippet,
                    "title": title,
                    "link": link,
                    "source": source
                })

    return snippet_facts

# Find best match using similarity on phrases
def find_best_matches(report_facts, snippet_facts, threshold=0.4):
    matches = []

    for rf in report_facts:
        best_match = None
        best_score = 0

        for sf in snippet_facts:
            # Match by numeric value
            if rf["value"] in sf["value"] or sf["value"] in rf["value"]:
                score = SequenceMatcher(None, rf["phrase"], sf["phrase"]).ratio()
                if score > best_score:
                    best_score = score
                    best_match = sf

        if best_match and best_score > threshold:
            matches.append({
                "fact": rf["phrase"],
                "match_score": round(best_score, 3),
                "source": best_match["source"],
                "title": best_match["title"],
                "snippet": best_match["snippet"],
                "link": best_match["link"]
            })

    return matches

# Run matching
report_facts = extract_facts_with_context(full_report)
snippet_facts = extract_snippet_facts(
    client_industry_summary + macro_news_summary +
    [a for articles in result.get("client_holdings_summary", {}).values() for a in articles]
)

matches = find_best_matches(report_facts, snippet_facts)

# Display results
for m in matches:
    print(f"Fact: {m['fact']}")
    print(f"Matched Snippet: {m['snippet']}")
    print(f"Match Score: {m['match_score']}")
    print(f"Source: {m['source']}")
    print(f"Title: {m['title']}")
    print(f"Link: {m['link']}")
    print("-" * 100)

Fact: broader context, Compass, Inc. (COMP) reported record first-quarter 2025 results, showcasing a 28.7% year-over-year revenue increase, reaching
Matched Snippet: ASML Holding N.V. ASML shares have plunged 8.6% since it reported first-quarter 2025 results on April 15. The stock has fallen largely due ...
Match Score: 0.441
Source: 
Title: ASML Holding Stock Down 9% Since Q1 Earnings - Yahoo Finance
Link: https://finance.yahoo.com/news/asml-holding-stock-down-9-190000037.html
----------------------------------------------------------------------------------------------------
Fact: (COMP) reported record first-quarter 2025 results, showcasing a 28.7% year-over-year revenue increase, reaching $1.4 billion. This strong
Matched Snippet: Revenue in Q1 2025 increased by 28.7% year-over-year to $1.4 billion as transactions increased 27.8% compared to the market transactions, which ...
Match Score: 0.464
Source: 
Title: Compass, Inc. Reports Record First Quarter 2025 Results
Link: https://fi